## Fix the coordsystem.json files generated by ESS.

This notebook replaces the `coordsystem.json` files of ESS with those generated by EEGLAB BidsTools
for the BCIT Mind Wandering data.


In [1]:
import os
import json
from hed.tools import BidsTsvDictionary, HedLogger
from hed.util import get_file_list, get_new_dataframe, get_value_dict, replace_values

# Variables to set for the specific dataset
bids_root_path = 's:/bcit/MindWanderingWorkingPhaseTwo'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
log_file_name = 'bcit_mind_wandering_06_fix_channels_log.json'

# Construct the list of .event files
logger = HedLogger()
channel_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_channels", exclude_dirs=exclude_dirs)
desc_dict = {'LN': 'Lane deviation from center line in meters.',
             'ANG': 'Steering wheel deviation angle in degrees.',
             'SP': 'Vehicle speed in mph.',
             'SD': 'Vehicle speed deviation.'}

for file in channel_files:
    df = get_new_dataframe(file)
    basename = os.path.basename(file)
    logger.add(basename, f"Created a dataframe for {basename}")
    channel_mask = df['name'].map(str).isin(['LN', 'ANG', 'SP', 'SD'])
    if sum(channel_mask) == 0:
        continue
    names = df.loc[channel_mask, 'name']
    df.loc[channel_mask, 'type'] = 'OTHER'
    df.loc[channel_mask, 'units'] = 'n/a'
    df.loc[channel_mask, 'sampling_frequency'] = 1024
    for index, name in names.iteritems():
        df.loc[index, 'description'] = desc_dict[name]
        logger.add(basename, f"Replacing {name} row with {str([name, 'OTHER', 'n/a', 1024, desc_dict[name]])}")
    df.to_csv(file, sep='\t', index=False)
    logger.add(basename, f"Saved as {basename}")

# Output and save the log
logger.print_log()
save_path = os.path.join(bids_root_path, 'code')
logger.save_log(save_path, log_name=log_file_name)

# Output errors
print("\nERROR Summary:")
logger.print_log(level="ERROR")

sub-01_ses-01_task-DriveWithTaskAudio_run-1_channels.tsv:
	[ Created a dataframe for sub-01_ses-01_task-DriveWithTaskAudio_run-1_channels.tsv]
	[ Replacing LN row with ['LN', 'OTHER', 'n/a', 1024, 'Lane deviation from center line in meters.']]
	[ Replacing ANG row with ['ANG', 'OTHER', 'n/a', 1024, 'Steering wheel deviation angle in degrees.']]
	[ Replacing SP row with ['SP', 'OTHER', 'n/a', 1024, 'Vehicle speed in mph.']]
	[ Replacing SD row with ['SD', 'OTHER', 'n/a', 1024, 'Vehicle speed deviation.']]
	[ Saved as sub-01_ses-01_task-DriveWithTaskAudio_run-1_channels.tsv]
sub-01_ses-01_task-DriveWithTaskAudio_run-2_channels.tsv:
	[ Created a dataframe for sub-01_ses-01_task-DriveWithTaskAudio_run-2_channels.tsv]
	[ Replacing LN row with ['LN', 'OTHER', 'n/a', 1024, 'Lane deviation from center line in meters.']]
	[ Replacing ANG row with ['ANG', 'OTHER', 'n/a', 1024, 'Steering wheel deviation angle in degrees.']]
	[ Replacing SP row with ['SP', 'OTHER', 'n/a', 1024, 'Vehicle speed in m